In [51]:
! pip install scikit-video
import os
import tensorflow as tf	
import numpy as np
import time
from tf_utils import input_fn_from_dataset,input_fn_frame_from_dataset,save_tf_record,prob_positive_class_from_prediction
from get_data import get_videos_from_folder,get_target_from_csv
from utils import save_solution
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
                                        MaxPooling2D)
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger

In [23]:
train_folder = os.path.join("../train/")
test_folder = os.path.join("../test/")

train_target = os.path.join('../train_target.csv')
my_solution_file = os.path.join('../solution.csv')

tf_record_dir = os.path.join('..','tf_records')
os.makedirs(tf_record_dir, exist_ok=True)

tf_record_train = os.path.join(tf_record_dir, 'train' + '.tfrecords')
tf_record_test = os.path.join(tf_record_dir, 'test' + '.tfrecords')
x_train = get_videos_from_folder(train_folder)
y_train = get_target_from_csv(train_target)
X = np.zeros((x_train.shape[0], 212, 100, 100,1))
for i in range(x_train.shape[0]):
    X[i,:x_train[i].shape[0],:x_train[i].shape[1],:x_train[i].shape[2],0] = x_train[i]

IndexError: tuple index out of range

In [60]:
#X = np.zeros(( 158,212, 100, 100,1))
#for i in range(158):
    #X[i,:x_train[i].shape[0],:x_train[i].shape[1],:x_train[i].shape[2],0] = x_train[i]


In [ ]:
batchsize_video = 1
nb_epoch=100
checkpointer = ModelCheckpoint( filepath=os.path.join('data', 'checkpoints', 'lcrn' + '-' + \
            '.{epoch:03d}-{val_loss:.3f}.hdf5'),
        verbose=1,
        save_best_only=True)

    
tb = TensorBoard(log_dir=os.path.join('data', 'logs', 'lcrn'))
early_stopper = EarlyStopping(patience=5)
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('data', 'logs', 'lcrn' + '-' + 'training-' + str(timestamp) + '.log'))
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2),activation='relu',padding='same'), input_shape=(212, 100, 100,1)))
model.add(TimeDistributed(Conv2D(32, (3,3),
kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False, dropout=0.5))
model.add(Dense(2, activation='softmax'))
metrics = ['accuracy']
optimizer = Adam(lr=1e-5, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
                       metrics=metrics)
model.fit(X, y_train, batch_size=32,  verbose=1,callbacks=[tb, early_stopper, csv_logger], epochs=nb_epoch)




Epoch 1/100
158/158 [==============================] - 477s 3s/step - loss: 0.6937 - acc: 0.4937
Epoch 2/100


/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


158/158 [==============================] - 480s 3s/step - loss: 0.6930 - acc: 0.5000
Epoch 3/100
158/158 [==============================] - 475s 3s/step - loss: 0.6923 - acc: 0.5000
Epoch 4/100
128/158 [=======================>......] - ETA: 1:31 - loss: 0.6908 - acc: 0.5078

In [48]:
x_test = get_videos_from_folder(test_folder)

array([1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0])

In [ ]:
X_test = np.zeros((x_test.shape[0], 212, 100, 100,1))
for i in range(x_test.shape[0]):
    X_test[i,:x_test[i].shape[0],:x_test[i].shape[1],:x_test[i].shape[2],0] = x_test[i]

In [ ]:
! pip install pandas
train_predictions = model.predict(X_test) 
dummy_solution = prob_positive_class_from_prediction(pred)
save_solution(my_solution_file,dummy_solution)